In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
import time

In [2]:
#chromebrowser set up

executable_path = {"executable_path": "chromedriver"}
browser =  Browser("chrome", **executable_path, wait_time=30, fullscreen=False, incognito=True, headless=False)

In [3]:
#URL's to scrape
url1 = 'https://mars.nasa.gov/news/'
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url3 = 'https://twitter.com/marswxreport?lang=en'
url4 = 'http://space-facts.com/mars/'
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

**************************************************************************
NASA Mars News
Scrape https://mars.nasa.gov/news/ and get the latest news title and text
**************************************************************************

In [4]:
#Nasa Mars News
browser.visit(url1)

In [5]:
# Retrieve page with the requests module
response = browser.html
soup = bs(response, 'html.parser')


In [6]:
# Retrieve the latest element that contains news title and news_paragraph
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA to Host Media Call on Next Mars Landing Site
NASA will host a media teleconference at 9 a.m. PST (noon EST) Monday, Nov. 19, to provide details about the Mars 2020 rover’s landing site on the Red Planet.


**************************************************************************
JPL Mars Space Images - Featured Image
Scrape https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars and get the link for the larget carousel image
**************************************************************************

In [7]:

browser.visit(url2)
f1 = browser.find_by_id("full_image")

f1.click()
time.sleep(1)
browser.click_link_by_partial_text('more info')

In [8]:
# Retrieve page with the requests module
response2 = browser.html
soup2 = bs(response2, 'html.parser')

In [9]:
#main_url = main one for in front
main_url = 'https://www.jpl.nasa.gov'
page_url = soup2.find(class_='lede').find('a').get('href')
featured_image_url = main_url + page_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18328_hires.jpg


**************************************************************************
Mars Weather
Scrape https://twitter.com/marswxreport?lang=en and get and save the westher tweets
**************************************************************************

In [10]:
# Retrieve page with the requests module to get Mars Weather
response3 = requests.get(url3)
soup3 = bs(response3.text, 'html.parser')


In [11]:
mars_weather_find = soup3.find_all('div', class_='js-tweet-text-container')
mars_weather = mars_weather_find[0].p.text
print(mars_weather)


Sol 2230 (2018-11-14), high -5C/23F, low -72C/-97F, pressure at 8.59 hPa, daylight 06:22-18:39


**************************************************************************
Mars Facts
Scrape https://space-facts.com/mars/ and get and save the facts to an HTML table string 

**************************************************************************

In [12]:
#Get Mars Facts
tables = pd.read_html(url4)[0]
tables.columns = ('fact', 'figure')
tables.set_index('fact', inplace=True)
tables

,figure
fact,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [13]:
#Put Mars facts in HTML table string
table_string = tables.to_html()
print(table_string)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>figure</th>
    </tr>
    <tr>
      <th>fact</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


**************************************************************************
Mars Hemispheres
Scrape https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars and get and save the high resolution images for each of Mars hemispheres

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.
**************************************************************************

In [14]:
# Visit and get Mars Hemispheres information - URL and title of each Hemisphere
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls
hemisphere_image_urls = []

# Store the main_ul
main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items:
   # Store title
   title = i.find('h3').text.strip('Enhanced')

   # Store link that leads to full image website
   partial_img_url = i.find('a', class_='itemLink product-item')['href']

   # Visit the link that contains the full image website
   browser.visit(main_url + partial_img_url)

   # HTML Object
   partial_img_html = browser.html

   # Parse HTML with Beautiful Soup
   soup = bs( partial_img_html, 'html.parser')

   # Retrieve full image source
   img_url = main_url + soup.find('img', class_='wide-image')['src']

   # Append to dictionaries
   hemisphere_image_urls.append({"title" : title, "img_url" : img_url})


# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [15]:
mars = []
mars.append({"news_title" : news_title, "news_short" : news_p, "featured_image_url" : featured_image_url,
            "mars_weather" : mars_weather, "mars_facts" : table_string, "hemispheres_urls" : hemisphere_image_urls})
mars

[{'news_title': 'NASA to Host Media Call on Next Mars Landing Site',
  'news_short': 'NASA will host a media teleconference at 9 a.m. PST (noon EST) Monday, Nov. 19, to provide details about the Mars 2020 rover’s landing site on the Red Planet.',
  'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18328_hires.jpg',
  'mars_weather': 'Sol 2230 (2018-11-14), high -5C/23F, low -72C/-97F, pressure at 8.59 hPa, daylight 06:22-18:39',
  'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>figure</th>\n    </tr>\n    <tr>\n      <th>fact</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobo